In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

#from mri_3d import *

from config_manager import get_directory

import pandas as pd
import numpy as np

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
from scipy import stats
import numpy as np
from keras.models import Model, load_model
import pandas as pd
import pickle
import h5py
from sklearn.decomposition import PCA

from data_io import ScanDataGenerator
from config_manager import get_directory

Using TensorFlow backend.


In [3]:
def scan_batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def extract_features_mri(path_to_images='/mnt/30T/adni/ADNI_MPRAGE_VENTRICLES.h5', path_to_model=None, feature_space_layer='dense_1', h5_dict=get_directory.adni_mprage_dict, X_ids=None, Y_ids=None, batch_sizes = 10, scan_dim=(96,96,96), crops=((0, 0), (0, 0), (0, 0)), zoom=1.0):
    # feature extraction
    
    #load model
    model = load_model(path_to_model)
    #remove output layer
    new_model = Model(model.inputs, model.get_layer(feature_space_layer).output)
    #load the weights
    new_model.set_weights(model.get_weights())
    
    # loading images using the dictionary to find the scan given an Image_ID:
    with open(h5_dict, 'rb') as pk:
        imgid = pickle.load(pk)
        
    img = ScanDataGenerator(path_to_images,img_id=X_ids,dataframe=get_directory.adni_dataframe, crop=crops
                                 ,input_dim=scan_dim
                                 ,img_to_i=imgid,zooms=zoom)
    #img_x, img_x = img
    #np_img = np.array(img)
    #np_img = np_img[:,0,0,:,:,:,:]
    #np_img = np_img.reshape((len(X_ids), scan_dim[0],scan_dim[1],scan_dim[2], 1))
    output_X = new_model.predict_generator(img, workers=6, use_multiprocessing=True, verbose=True) #images should have the dimensions: (batch_size,scan_dim[0],scan_dim[1],scan_dim[2],1)
    
    img = ScanDataGenerator(path_to_images,img_id=Y_ids,dataframe=get_directory.adni_dataframe, crop=crops
                                 ,input_dim=scan_dim
                                 ,img_to_i=imgid,zooms=zoom)
    #img_y, img_y = img
    #np_img = np.array(img)
    #np_img = np_img[:,0,0,:,:,:,:]
    #np_img = np_img.reshape((len(Y_ids), scan_dim[0],scan_dim[1],scan_dim[2], 1)) 
    output_Y = new_model.predict_generator(img, workers=6, use_multiprocessing=True, verbose=True) #images should have the dimensions: (batch_size,scan_dim[0],scan_dim[1],scan_dim[2],1)
    
    feats_X = np.array(output_X)
    feats_Y = np.array(output_Y)

    # end feature extraction
    return feats_X.astype(np.float128), feats_Y.astype(np.float128)


def compute_p_value(feats_X, feats_Y, n_feats=10):
    n, d = feats_X.shape
    m = len(feats_Y)
    pca = PCA(n_components=n_feats)
    feats = np.vstack((feats_X, feats_Y))
    feats = pca.fit_transform(feats)
    feats_X, feats_Y = feats[:n], feats[n:]
    d = n_feats
    mean_fX = feats_X.mean(0)
    mean_fY = feats_Y.mean(0)
    D = mean_fX - mean_fY

    eps_ridge = 1e-8    # add ridge to Covariance for numerical stability
    all_features = np.concatenate([feats_X, feats_Y])
    Cov_D = (1./n + 1./m) * np.cov(all_features.T) + eps_ridge * np.eye(d)

    statistic = D.dot(np.linalg.solve(Cov_D, D))
    p_val = 1. - stats.chi2.cdf(statistic, d)
    return p_val

def compute_p_value_mmd(feats_X, feats_Y, n_permute=10000):
    stat = compute_mmd(feats_X, feats_Y)
    n, m = len(feats_X), len(feats_Y)
    l = n + m
    feats_Z = np.vstack((feats_X, feats_Y))

    resampled_vals = np.empty(n_permute)
    for i in range(n_permute):
        index = np.random.permutation(l)
        feats_X, feats_Y = feats_Z[index[:n]], feats_Z[index[n:]]
        resampled_vals[i] = compute_mmd(feats_X, feats_Y)
    p_val = np.mean(stat < resampled_vals)
    return p_val
def compute_mmd(X, Y):
    mean_X = X.mean(0)
    mean_Y = Y.mean(0)
    D = mean_X - mean_Y
    stat = np.linalg.norm(D)**2
    return stat

In [8]:
adni = pd.read_csv('/mnt/30T/adni/adni_mprage_all.csv')

normals_first_scans = adni[adni.DX_Group == 'Normal'].drop_duplicates(subset='Subject_ID')

mcis_first_scans = adni[adni.DX_Group == 'MCI'].drop_duplicates(subset='Subject_ID')

In [9]:
x,y = extract_features_mri(path_to_images='/mnt/30T/adni/ADNI_MPRAGE_ALL.h5',
    path_to_model="/home/Shahryar.Khorasani/adni/models/ukb_reg_age_880902_model.h5",
    feature_space_layer='dense_1',
    h5_dict='/mnt/30T/adni/ADNI_MPRAGE_ALL_DICT.pickle',
    X_ids=normals_first_scans.Image_ID,
    Y_ids=mcis_first_scans.Image_ID,
    batch_sizes=2,
    scan_dim=(256, 256, 256),
    crops=((48, 48), (32, 32),(48, 48)),
    zoom=1.0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
287/287 [==============================] - 4325s 15s/step


In [10]:
import math
def get_n_features(x,y):
    n = np.int(np.round(math.sqrt((x.shape[0]+y.shape[0])/2)))
    return n 

In [11]:
p_normal_mci = compute_p_value(x,y,n_feats=get_n_features(x,y))

In [12]:
p_normal_mci

2.3385760117289323e-07

In [14]:
ad_first_scans = adni[adni.DX_Group == 'AD'].drop_duplicates(subset='Subject_ID')

smc_first_scans = adni[adni.DX_Group == 'SMC'].drop_duplicates(subset='Subject_ID')

In [16]:
len(smc_first_scans)

76

In [ ]:
ad,smc = extract_features_mri(path_to_images='/mnt/30T/adni/ADNI_MPRAGE_ALL.h5',
    path_to_model="/home/Shahryar.Khorasani/adni/models/ukb_reg_age_880902_model.h5",
    feature_space_layer='dense_1',
    h5_dict='/mnt/30T/adni/ADNI_MPRAGE_ALL_DICT.pickle',
    X_ids=ad_first_scans.Image_ID,
    Y_ids=smc_first_scans.Image_ID,
    batch_sizes=2,
    scan_dim=(256, 256, 256),
    crops=((48, 48), (32, 32),(48, 48)),
    zoom=1.0)

In [18]:
p_ad_nromal = compute_p_value(ad,x,n_feats=get_n_features(ad,x))

In [19]:
p_ad_nromal

1.8494394238288692e-08

In [20]:
p_ad_mci = compute_p_value(ad,y, n_feats=get_n_features(ad,y))

In [21]:
p_ad_mci

0.005271691403107681

In [4]:
len(ad)

NameError: name 'ad' is not defined